In [10]:
#!tar xvf Traindata.tar

Traindata/
Traindata/18/
Traindata/18/.DS_Store
Traindata/18/IMG_0045.jpg
Traindata/18/IMG_0274.jpg
Traindata/18/IMG_1073.jpg
Traindata/18/IMG_1074.jpg
Traindata/18/IMG_1213.jpg
Traindata/18/IMG_1230.jpg
Traindata/18/IMG_1319.jpg
Traindata/18/IMG_1320.jpg
Traindata/18/IMG_1325.jpg
Traindata/18/IMG_1491.jpg
Traindata/18/IMG_1513.jpg
Traindata/18/IMG_1514.jpg
Traindata/18/IMG_1517.jpg
Traindata/18/IMG_1541.jpg
Traindata/18/IMG_1547.jpg
Traindata/18/IMG_1721.jpg
Traindata/18/IMG_2368.jpg
Traindata/18/IMG_2726.jpg
Traindata/18/IMG_2768.jpg
Traindata/18/IMG_2874.jpg
Traindata/18/IMG_2930.jpg
Traindata/18/IMG_3040.jpg
Traindata/18/IMG_3068.jpg
Traindata/18/IMG_3201.jpg
Traindata/18/IMG_3237.jpg
Traindata/18/IMG_3266.jpg
Traindata/18/IMG_3574.jpg
Traindata/18/IMG_3764.jpg
Traindata/18/IMG_4134.jpg
Traindata/18/IMG_4762.jpg
Traindata/18/IMG_5071.jpg
Traindata/18/IMG_5098.jpg
Traindata/18/IMG_5181.jpg
Traindata/18/IMG_5668.jpg
Traindata/18/IMG_5751.jpg
Traindata/18/IMG_6174.jpg
Traindata/18/IMG

In [33]:
import argparse
import pickle

# 1.Import / 設定路徑 / Define Class

In [1]:
import os,shutil
import glob
import numpy as np

from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.applications import Xception


#Read File
Train_dir = r'./Traindata/'
Test_dir = r'./test_datasets/'

In [2]:
Train_dir 

'./Traindata/'

1.1 - Set Class Map

In [ ]:
class_mapping = {
 'lala': 5, 
 'Jerry': 9,
 '18': 0,
 'tim': 7,
 'tan': 6,
 'wwk': 8,
 'Prof.Kang': 2,
 'Yoshit': 4,
 'Rockman': 3,
 'Ellie': 1}

class_mapping 

{'lala': 5,
 'Jerry': 9,
 '18': 0,
 'tim': 7,
 'tan': 6,
 'wwk': 8,
 'Prof.Kang': 2,
 'Yoshit': 4,
 'Rockman': 3,
 'Ellie': 1}

# 2. Built Xception Model

### Sequential : keras的序慣模型 model = models.Sequential()

- 將捲積基底(convolution base)視為一層加入Sequential 模型中  :   model.add(convolution_base) 
- model.add(layers.GlobalAveragePooling2D(data_format = 'channels_last'))
- dropout
- Set Output :  "Class & Softmax" :  

In [ ]:
# Set Convolution Base
convolution_base = Xception(weights = 'imagenet',
                      include_top = False,
                        input_shape =(299,299,3))


model = models.Sequential()
model.add(convolution_base) #將捲積基底視為層加入Sequential 模型中
model.add(layers.GlobalAveragePooling2D(data_format = 'channels_last'))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(10, activation = 'softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
convolution_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

Data Augmentation : 
    
##### 餐考資料 : https://zhuanlan.zhihu.com/p/30197320
    
1. 從整個Dataset將圖片標準化: 
   featurewise_center=True,  
   featurewise_std_normalization=True ,  
   
2. 從自身圖片標準化: 
   samplewise_center=True, 
   
3. 保留最重要特徵:
   zca_whitening=True,
   
4. 旋轉圖片角度: 
   rotation_range = 30,
   
5. 左右長度平移 / 上下長度平移
   width_shift_range =0.2, 
   height_shift_range = 0.2,
   
6. 圖形各種梯形轉換
   hear_range = 0.3,
   
7. 圖片大小縮放
   zoom_range = 0.3, 
   
8. 圖片色階轉換
   channel_shift_range = 10
   
9. 隨機圖片水平翻轉 / 隨機圖片垂直顛倒
   horizontal_flip = True, vertical_flip = True,
   
10. 將像素質縮小以避免落入activation function之死亡區
    rescale = 1./255, 
    
11. fill_mode = 'constant'  or ( “reflect”、“wrap”、“nearest”)
    fill_mode为填充模式，如前面提到，当对图片进行平移、放缩、错切等操作时，图片中会出现一些缺失的地方，那这些缺失的地方该用什么方式补全呢？就由fill_mode中的参数确定，包括：“constant”、“nearest”（默认）、“reflect”和“wrap”。这四种填充方式的效果对比如图18所示，从左到右，从上到下分别为：“reflect”、“wrap”、“nearest”、“constant”。

12. preprocessing_function = myFunc, 
           

In [ ]:
#Import train 好的 Xception Model (路徑在調整)
top_weights_path = os.path.join(os.path.abspath('./models'), 'Xception_top_model_weight_v5.h5')

#設定執行狀況回復指令 & Earlystop
callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_acc', patience= 5, verbose=0)
    ]

def myFunc(image):
    image = np.array(image)
    image = image - np.array([123.68,116.779,103.939])
    image = image[:,:,::-1]
    return image


train_datagen = ImageDataGenerator(
            
            preprocessing_function = myFunc, 
            featurewise_center=True,  
            featurewise_std_normalization=True , #從整個Dataset將圖片標準化 
            samplewise_center=True, #從自身圖片標準化
            zca_whitening=True, # 保留最重要特徵
          
            rotation_range = 10, #顧名思義旋轉圖片角度
            width_shift_range =0.1, #左右長度平移
            height_shift_range = 0.1, #上下長度平移
            shear_range = 0.1, #圖形各種梯形轉換
            zoom_range = [0.2,1], #圖片大小縮放
            horizontal_flip = True, #隨機圖片水平翻轉
            vertical_flip = True, #隨機圖片垂直顛倒
            channel_shift_range = 20, #圖片色階轉換
            rescale = 1./255,   #將像素質縮小以避免落入activation function之死亡區
    
            brightness_range=[0,1],
            fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale = 1./255,
                                   preprocessing_function = myFunc) 


train_generator = train_datagen.flow_from_directory(
                Train_dir,
                target_size = (299,299),
                batch_size = 15,
                class_mode = 'categorical')

Test_generator = test_datagen.flow_from_directory(
                    Test_dir,
                    target_size = (299,299),
                    batch_size = 12,
                    class_mode = 'categorical')

model.compile(loss = 'categorical_crossentropy',
             optimizer = optimizers.Adam(lr = 0.0001),
                metrics = ['acc'])

C:\Users\user\anaconda3new\lib\site-packages\keras_preprocessing\image\image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Found 536 images belonging to 10 classes.
Found 132 images belonging to 10 classes.


In [ ]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch = 100,
            epochs = 25,
            validation_data = Test_generator,
            validation_steps = 13,
            callbacks = callbacks_list)

C:\Users\user\anaconda3new\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\user\anaconda3new\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/25


C:\Users\user\anaconda3new\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\user\anaconda3new\lib\site-packages\keras_preprocessing\image\image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 1/10 [==>...........................] - ETA: 10:30 - loss: 2.4838 - acc: 0.0000e+00

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)


plt.plot(epochs, acc, 'b', label = 'Training acc')
plt.plot(epochs, val_acc, 'b',label = 'Validation acc',color = 'red')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()


plt.plot(epochs, loss, 'b', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss',color = 'red')
plt.title('Training and validation loss')

plt.show()

In [14]:
from sklearn.metrics import classification_report, confusion_matrix


validation_generator = test_datagen.flow_from_directory(
                    Test_dir,
                    target_size = (299, 299),
                    batch_size = 15,
                    shuffle = False,
                    class_mode = 'categorical')

model.evaluate_generator(validation_generator,verbose=1)

from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_generator,verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = list(validation_generator.class_indices.keys())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Found 132 images belonging to 10 classes.
9/9 [==============================] - 1s 135ms/step
Confusion Matrix
[[13  0  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  1  0  0  0]
 [ 0  0  1  2  0  0  0  0  6  0]
 [ 0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  1  0]
 [ 0  1  0  0  0  0 11  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  1  0  1 10  1]
 [ 0  0  0  0  0  0  0  1  0 17]]
Classification Report
              precision    recall  f1-score   support

          18       1.00      1.00      1.00        13
       Ellie       0.92      0.92      0.92        12
       Jerry       1.00      0.11      0.20         9
   Prof.Kang       0.89      1.00      0.94        16
     Rockman       1.00      1.00      1.00        17
      Yoshit       0.92      0.92      0.92        12
        lala       0.92      0.92      0.92        12
         tan       0.83      1.00      0.91        10
         tim       0.59      0.77      0.67     

In [15]:
import cv2
for x in range(len(validation_generator.filenames)):
    if y_pred[x] != validation_generator.classes[x]:
        print('Wrong Name : ',validation_generator.filenames[x])
        print('Actually', validation_generator.classes[x])
        print('Machine Predict',y_pred[x])
        print(" ")

Wrong Name :  Ellie/IMG_0042.jpg
Actually 1
Machine Predict 6
 
Wrong Name :  Jerry/IMG_1496.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Jerry/IMG_1525.jpg
Actually 2
Machine Predict 3
 
Wrong Name :  Jerry/IMG_1526.jpg
Actually 2
Machine Predict 3
 
Wrong Name :  Jerry/S__27599002.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Jerry/S__27599004.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Jerry/S__27599005.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Jerry/S__27599006.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Jerry/S__27599018.jpg
Actually 2
Machine Predict 8
 
Wrong Name :  Yoshit/IMG_1544.jpg
Actually 5
Machine Predict 8
 
Wrong Name :  lala/IMG_9539.jpg
Actually 6
Machine Predict 1
 
Wrong Name :  tim/00100lPORTRAIT_00100_BURST20181201085911494_COVER.jpg
Actually 8
Machine Predict 5
 
Wrong Name :  tim/IMG_20191105_115022.jpg
Actually 8
Machine Predict 9
 
Wrong Name :  tim/IMG_9459.jpg
Actually 8
Machine Predict 7
 
Wrong Name :  wwk/IMG_1550.jpg
Actually 9
M

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 10
img_gen = ImageDataGenerator(  
        rotation_range=50,
        width_shift_range=0.3,
        height_shift_range=0.5,
        shear_range=0.2,
        zoom_range=0.5,
        horizontal_flip=True,
        fill_mode='constant',
        cval=0)

for i in range(len(validation_generator.filenames)):
    if y_pred[x] != validation_generator.classes[x]:
        
        
        print('Wrong Name : ',validation_generator.filenames[x])
        print('Actually', validation_generator.classes[x])
        print('Machine Predict',y_pred[x])
        print(" ")
        

# Save Model as...

In [33]:
#from tqdm import tqdm
import glob
import cv2
import numpy as np
img_paths = glob.glob('./test_datasets_v2/*.jpg')
images = []
img_sizes = []
ids = []
for img_path in img_paths:
    ids.append(img_path.split("/")[-1].split(".")[0])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (299, 299))
    img = np.array(img)
    img = img - np.array([123.68,116.779,103.939])
    img = img[:,:,::-1]
    if img.shape not in img_sizes:
        img_sizes.append(img.shape)

    images.append(img)

images_1 = np.array(images)
x_Test4D = images_1.reshape(images_1.shape[0],299,299,3).astype('float32') 
x_Test4D_normalize = x_Test4D/255

In [34]:
model.save('./MYmodels/Xception7.h5')
model.predict_classes
predict_value = model.predict_classes(x_Test4D_normalize)

In [35]:
model.predict_classes
predict_value = model.predict_classes(x_Test4D_normalize)
predict_value

array([1, 7, 4, 4, 4, 4, 9, 9, 7, 3, 9, 8, 9, 0, 8, 4, 4, 4, 6, 9, 5, 3,
       0, 9, 3, 0, 4, 5, 5, 8, 3, 0, 6, 9, 4, 9, 3, 3, 6, 3, 6, 0, 0, 9,
       5, 1, 4, 9, 1, 4, 8, 6, 5, 0, 3, 0, 8, 8, 5, 0, 0, 1, 9, 5, 3, 2,
       6, 1, 6, 0, 4, 4, 6, 5, 5, 3, 4, 3, 5, 7, 9, 6, 5, 0, 6, 6, 5, 7,
       3, 5, 4, 8, 1, 6, 9, 3, 8, 8, 5, 5, 7, 3, 9, 9, 7, 6, 0, 7, 4, 5,
       8, 8, 1, 5, 6, 8, 4, 3, 0, 9, 9, 8, 9, 6, 3, 8, 7, 1, 0, 1, 7, 0])

In [66]:
predict_value = model.predict_classes(x_Test4D_normalize)

import numpy as np
z = np.array([1,2])
z[1] = 6

In [28]:
# 1.將數值依照順序排列 才可設迴圈

# 2.迴圈要怎麼設?
    # a.將數值依照順序排列 : np.arange(len(predict_value)):
    # b.用for依序跑array : 
    # c.設判斷式 : 判斷甚麼? ->判斷predict_value裡面的值 並用它自己(predict_value)取代原先判斷式的值

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131])

In [39]:
import numpy as np 
np.arange(len(predict_value)) #a

for i in np.arange(len(predict_value)):
    if predict_value[i] == 1 : 
        predict_value[i] = 2

array([2, 7, 4, 4, 4, 4, 9, 9, 7, 3, 9, 8, 9, 0, 8, 4, 4, 4, 6, 9, 5, 7,
       0, 9, 3, 0, 4, 5, 5, 8, 3, 0, 6, 9, 4, 9, 3, 3, 6, 2, 6, 7, 0, 9,
       5, 2, 4, 9, 2, 4, 8, 6, 5, 0, 3, 0, 9, 8, 5, 0, 0, 2, 9, 5, 3, 3,
       2, 2, 6, 0, 4, 4, 6, 5, 5, 3, 4, 3, 5, 7, 9, 6, 5, 0, 6, 6, 5, 7,
       3, 5, 4, 8, 2, 6, 9, 3, 8, 8, 5, 4, 7, 3, 9, 9, 7, 2, 0, 7, 4, 4,
       8, 8, 2, 5, 6, 8, 4, 3, 0, 9, 9, 4, 9, 6, 3, 8, 7, 2, 0, 2, 7, 7])

In [54]:
type(class_mapping)

dict

In [57]:
classes
classes[1]

2

In [17]:
predict_value = model.predict_classes(x_Test4D_normalize)

import numpy as np 
np.arange(len(predict_value))
classes = np.array([0,1,2,3,4,5,6,7,8,9])


# for l in classes: 
for i in np.arange(len(predict_value)): 
          if predict_value[i] == classes[2] : predict_value[i] = classes[9]
          elif predict_value[i] == classes[3] : predict_value[i] = classes[5]
          elif predict_value[i] == classes[4] : predict_value[i] = classes[2]
          elif predict_value[i] == classes[5] : predict_value[i] = classes[3]
          elif predict_value[i] == classes[9] : predict_value[i] = classes[4]
          else: predict_value[i] = predict_value[i]
            
predict_value

array([1, 7, 2, 2, 2, 2, 4, 4, 7, 5, 4, 8, 4, 0, 8, 2, 2, 2, 6, 4, 3, 7,
       0, 4, 5, 0, 2, 3, 3, 8, 5, 0, 6, 4, 2, 4, 5, 5, 6, 5, 6, 7, 7, 4,
       3, 1, 2, 4, 1, 2, 8, 6, 3, 0, 5, 0, 4, 8, 3, 0, 0, 1, 4, 3, 5, 9,
       1, 1, 6, 0, 2, 2, 6, 3, 3, 5, 2, 5, 8, 7, 4, 6, 3, 0, 6, 6, 3, 7,
       5, 3, 2, 8, 1, 6, 4, 5, 8, 8, 3, 5, 7, 5, 4, 4, 7, 1, 0, 7, 2, 5,
       8, 8, 1, 3, 6, 8, 2, 5, 0, 4, 4, 8, 4, 6, 5, 8, 7, 1, 0, 1, 7, 4])

In [67]:
predict_value

array([1, 7, 4, 4, 4, 4, 9, 9, 7, 3, 9, 8, 9, 0, 8, 4, 4, 4, 6, 9, 5, 7,
       0, 9, 3, 0, 4, 5, 5, 8, 3, 0, 6, 9, 4, 9, 3, 3, 6, 2, 6, 7, 0, 9,
       5, 1, 4, 9, 1, 4, 8, 6, 5, 0, 3, 0, 9, 8, 5, 0, 0, 1, 9, 5, 3, 3,
       1, 1, 6, 0, 4, 4, 6, 5, 5, 3, 4, 3, 5, 7, 9, 6, 5, 0, 6, 6, 5, 7,
       3, 5, 4, 8, 1, 6, 9, 3, 8, 8, 5, 4, 7, 3, 9, 9, 7, 1, 0, 7, 4, 4,
       8, 8, 1, 5, 6, 8, 4, 3, 0, 9, 9, 4, 9, 6, 3, 8, 7, 1, 0, 1, 7, 7])

In [85]:
np.arange(len(predict_value))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131])

In [93]:
for i in np.arange(len(predict_value)):   
    if predict_value[i] == 2: 
        predict_value[i] = 9
    elif predict_value[i] == 3: 
        predict_value[i] = 5
    elif predict_value[i] == 4: 
        predict_value[i] = 2
    elif predict_value[i] == 5: 
        predict_value[i] = 3
    elif predict_value[i] == 9:
        predict_value[i] = 4
    else:
        predict_value[i]=predict_value[i]
predict_value

array([1, 7, 2, 2, 2, 2, 4, 4, 7, 5, 4, 8, 4, 0, 8, 2, 2, 2, 6, 4, 8, 5,
       0, 4, 5, 0, 2, 3, 3, 8, 5, 0, 6, 4, 2, 4, 5, 5, 6, 5, 6, 7, 0, 4,
       3, 1, 2, 4, 1, 2, 8, 6, 3, 0, 5, 0, 5, 8, 3, 0, 0, 1, 4, 3, 5, 9,
       1, 1, 6, 0, 2, 2, 6, 3, 3, 5, 2, 5, 8, 7, 4, 1, 3, 0, 6, 6, 3, 7,
       5, 3, 2, 4, 1, 6, 4, 5, 8, 8, 3, 2, 7, 5, 4, 4, 7, 1, 0, 7, 2, 2,
       8, 5, 1, 3, 6, 8, 2, 5, 0, 4, 4, 8, 4, 6, 5, 8, 7, 1, 0, 1, 7, 7])

In [81]:
class_mapping

4
4
4


9
8
4


In [36]:
dict_new = {value:key for key,value in train_generator.class_indices.items()}
print(dict_new)

new = [dict_new.get(i, i) for i in predict_value] #轉換class

new2 = [class_mapping.get(i, i) for i in new]

{0: '18', 1: 'Ellie', 2: 'Jerry', 3: 'Prof.Kang', 4: 'Rockman', 5: 'Yoshit', 6: 'lala', 7: 'tan', 8: 'tim', 9: 'wwk'}


In [29]:
class_mapping

{'lala': 5,
 'Jerry': 9,
 '18': 0,
 'tim': 7,
 'tan': 6,
 'wwk': 8,
 'Prof.Kang': 2,
 'Yoshit': 4,
 'Rockman': 3,
 'Ellie': 1}

# Save to CSV.file & read file

In [37]:
from tensorflow.keras.models  import load_model
model = load_model('./MYmodels/Xception7.h5')

In [16]:
y_pred = mode

In [39]:
import pandas as pd
submit = pd.read_csv("sample_submition.csv")
submit = pd.DataFrame({"id":ids, "class":new2})
submit.to_csv('Xception7.csv', index= False)